In [4]:
import html2text


def html_to_plain_text(html_content):
    # Criar o conversor
    h = html2text.HTML2Text()
    
    # Configurações para remover formatação Markdown
    h.ignore_links = True   # Ignorar links
    h.ignore_images = True  # Ignorar imagens
    h.ignore_emphasis = True  # Ignorar negrito/itálico (sem asteriscos ou underlines)
    h.body_width = 0  # Não adicionar quebras de linha automáticas
    
    # Converter HTML para texto simples
    plain_text = h.handle(html_content)
    
    return plain_text

# Exemplo de uso:
html_content = '''
<div class="vitafor-store-theme-9-x-description"><div><p>
<div>
<p><span data-contrast="auto">Colagentek ® é um suplemento alimentar em pó à base de peptídeos de colágeno, com zero adição de açúcares e enriquecido com vitaminas e minerais.</span></p>
<p><strong><span data-contrast="auto">Pele viva e hidratada</span></strong></p>
<p><strong><span data-contrast="auto">Colagentek ® Vitafor é referência em colágeno</span></strong><span data-contrast="auto">, quanto a composição, sabor e praticidade no consumo.&nbsp; A combinação de 9g de colágeno hidrolisado por porção, associada a vitaminas e minerais, que auxiliam na formação do colágeno</span><span data-contrast="auto">1</span><span data-contrast="auto">, manutenção da pele e cabelo</span><span data-contrast="auto">2, 3</span><span data-contrast="auto">.</span></p>
<p>&nbsp;</p>
</div>
<p>&nbsp;</p>
<div>
<p><strong><span data-contrast="auto">Benefícios do produto:</span></strong></p>
<ul>
<li><span data-contrast="auto"> ²A biotina contribui para a manutenção do cabelo e da pele;</span></li>
<li><span data-contrast="auto"> ¹A vitamina C auxilia na formação do colágeno;</span></li>
<li><span data-contrast="auto"> A vitamina A auxilia na visão;</span></li>
<li><span data-contrast="auto"> As vitaminas A, B6, B12, C, D, ácido fólico e o cobre auxiliam no funcionamento do sistema imune;</span></li>
<li><span data-contrast="auto"> ³As vitaminas A e B3 contribuem para a manutenção da pele e o cobre para a pigmentação de cabelo e pele;</span></li>
<li><span data-contrast="auto"> As vitaminas B3, B5, B6, B12, C, biotina, o cobre e o manganês auxiliam no metabolismo energético;</span></li>
<li><span data-contrast="auto"> As vitaminas C e E, o cobre e o manganês são antioxidantes que auxiliam na proteção dos danos causados pelos radicais livres;</span></li>
<li><span data-contrast="auto"> Manganês auxilia na formação de ossos e a vitamina D na formação de ossos e dentes;</span></li>
<li><span data-contrast="auto"> A vitamina D auxilia no funcionamento muscular;</span></li>
<li><span data-contrast="auto"> Ácido fólico auxilia na síntese de aminoácidos;</span></li>
<li><span data-contrast="auto"> Cobre auxilia na manutenção dos tecidos conjuntivos e o manganês na manutenção dos tecidos conectivos</span></li>
</ul>
</div>
<p>&nbsp;</p>
</div></div>
'''

markdown = html_to_plain_text(html_content)
print(markdown)


Colagentek ® é um suplemento alimentar em pó à base de peptídeos de colágeno, com zero adição de açúcares e enriquecido com vitaminas e minerais.

Pele viva e hidratada

Colagentek ® Vitafor é referência em colágeno, quanto a composição, sabor e praticidade no consumo.  A combinação de 9g de colágeno hidrolisado por porção, associada a vitaminas e minerais, que auxiliam na formação do colágeno1, manutenção da pele e cabelo2, 3.

 

 

Benefícios do produto:

  * ²A biotina contribui para a manutenção do cabelo e da pele;
  * ¹A vitamina C auxilia na formação do colágeno;
  * A vitamina A auxilia na visão;
  * As vitaminas A, B6, B12, C, D, ácido fólico e o cobre auxiliam no funcionamento do sistema imune;
  * ³As vitaminas A e B3 contribuem para a manutenção da pele e o cobre para a pigmentação de cabelo e pele;
  * As vitaminas B3, B5, B6, B12, C, biotina, o cobre e o manganês auxiliam no metabolismo energético;
  * As vitaminas C e E, o cobre e o manganês são antioxidantes que auxili